In [ ]:
Port of the simhash algo (from a python port) in julia lang.

In [19]:
# run to reload the workspace/types and avoid the method overwrite warning
workspace()

In [20]:
using Nettle

In [21]:
_regex = r"[\w\u4e00-\u9fcc]+"

r"[\w\u4e00-\u9fcc]+"

In [1]:
type Simhash
    # this, unfortunately, does not hash on init
    value::UInt128
    f::Int64
end

function _hashfunc(x)
    h = Hasher("md5")
    update!(h, x)
    return parse(UInt128, hexdigest!(h), 16)
end

function distance(one::Simhash, another::Simhash)
    # this is ridiculous, julia
    x =  BigInt((one.value $ another.value) & ((UInt128(1) << UInt128(one.f)) - UInt128(1)))
    ans = 0
    while x > 0
        ans += 1
        x &= x - 1
    end
    return ans
end

function _tokenize(content::String)
    content = lowercase(content)
    content = join(match(_regex, content).captures(), '')
    return _slide(content)
end

function _slide(content::String, width=4)
    return [content[i:i + width - 1] for i in 1:max(length(content) - width, 1) + 1] 
end

function build_by_text(content::String)
    features = _tokenize(content)
    # unported!!
    fdict = {k:sum(1 for _ in g) for k, g in groupby(sort(features))}
    
    return build_by_features(features)
end

function build_by_features(features)
    
end

build_by_features (generic function with 0 methods)

In [ ]:
# port of the SimHashIndex class

type SimhashIndex 
    # from py : list of tuples
    objs
    f::Int64
    k::Float16
    
    count::Int16
    
    # from py : default dict of sets
    bucket 
end

function get_near_dups
    
end

function add_to_index
    
end

function delete_from_index
    
end


In [23]:
s1 = Simhash(_hashfunc("23456"), 64)
s2 = Simhash(_hashfunc("23766"), 64)

println(distance(s1, s2))



manual = 17988778886168394363
38
